In [189]:
# Imports
%matplotlib inline


from My_Functions import * 
import numpy as np
import pandas as pd
import seaborn as sns
import warnings

from collections import defaultdict

from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, r2_score,roc_curve
from sklearn.model_selection import train_test_split, KFold,StratifiedKFold
from sklearn.model_selection import cross_val_score, cross_val_predict,validation_curve
from sklearn.ensemble import RandomForestRegressor


from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelBinarizer, RobustScaler,PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor
from scipy import stats
from scipy.stats import skew, boxcox_normmax
from scipy.special import boxcox1p

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator , MultipleLocator
##from matplotlib import XAxis
from gplearn.genetic import SymbolicRegressor

from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, LabelBinarizer,MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import RFE, RFECV
from sklearn.linear_model import LogisticRegression,LinearRegression, OrthogonalMatchingPursuit
from sklearn.model_selection import train_test_split , TimeSeriesSplit, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
from matplotlib.gridspec import GridSpec
import plotly.tools as tls
import plotly
plotly.tools.set_credentials_file(username='Furqan92', api_key='22DfVN5rFRg79OYygN5h')
import plotly.plotly as py
from sklearn.decomposition import PCA
from pandas import DataFrame 
from sklearn.exceptions import ConvergenceWarning
import warnings

warnings.simplefilter(action='ignore')
random_seed = 1234
tscv = TimeSeriesSplit(n_splits=5)

In [190]:
# Load datasets
wd_h=pd.read_csv('./weekdays_data_prepared.csv')
we_h=pd.read_csv('./weekends_data_prepared.csv')
wd_h['dteday']=pd.to_datetime(wd_h['dteday'], format='%Y-%m-%d')
we_h['dteday']=pd.to_datetime(we_h['dteday'], format='%Y-%m-%d')

## Subset 2011 -> 2012Q3 Data

In [191]:
X_Train_2011_2012Q2 = wd_h[wd_h['dteday'] < datetime.datetime(2012, 7, 1, 0, 0)].drop(['cnt','casual','registered','dteday'],axis=1)## NONE OF THst_EM IN DATA
Y_cnt_train_2011_2012Q2 =wd_h['cnt'][wd_h['dteday'] < datetime.datetime(2012, 7, 1, 0, 0)]

X_Test_2012Q3 = wd_h[(wd_h['dteday'] >= datetime.datetime(2012, 7, 1, 0, 0)) & (wd_h['dteday'] <= datetime.datetime(2012, 9, 30, 0, 0))].drop(['cnt','casual','registered','dteday'],axis=1)## NONE OF THEM IN DATA
Y_cnt_test_2012Q3 =wd_h['cnt'][(wd_h['dteday'] >= datetime.datetime(2012, 7, 1, 0, 0)) & (wd_h['dteday'] <= datetime.datetime(2012, 9, 30, 0, 0))]

## Linear regression w/ GS for *Weekdays* for 2011 -> 2012Q3

In [192]:
##Linear Regression
lm_parameters = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True]}

lm = GridSearchCV(LinearRegression(),
                                 param_grid=lm_parameters,
                                 cv=tscv,return_train_score=True)

lm.fit(X_Train_2011_2012Q2, Y_cnt_train_2011_2012Q2)
# lm.cv_results_
lm_predictions = lm.predict(X_Test_2012Q3)
lm.score(X_Test_2012Q3, Y_cnt_test_2012Q3)

0.8585518889635224

## Random Forest w/ GS for *Weekdays* for 2011 -> 2012Q3

In [193]:
RF_parameters = {'n_estimators': [10, 30 ,100],
                                             'bootstrap': [True],
                                             'max_depth': [80, 100 ],
                                             'max_features': ['sqrt',16],
                                             'min_samples_leaf': [2,  5 , 8],
                                             'min_samples_split': [ 10 , 8 , 15],
                                            'random_state':[random_seed],
                                            'criterion':['mse']}
rf = GridSearchCV(RandomForestRegressor(),
                                 param_grid= RF_parameters,
                                 cv=tscv)

rf.fit(X_Train_2011_2012Q2, Y_cnt_train_2011_2012Q2)
# rf.cv_results_
rf_predictions = lm.predict(X_Test_2012Q3)
rf.score(X_Test_2012Q3, Y_cnt_test_2012Q3)

0.8854212997256523

## Taking predictions from LR and Random Forest to test on 2012Q4

In [194]:
# LR and Random Forest predictions
combinedPredictions = pd.DataFrame({'lm_predictions':lm_predictions,'rf_predictions':rf_predictions})

# Concat original data
X_Test_2012Q3.reset_index(drop=True,inplace=True)
combinedPredictions = pd.concat([combinedPredictions,X_Test_2012Q3], axis=1)

# Target data
combinedPredictionsTarget = pd.DataFrame({'target':Y_cnt_test_2012Q3})

#Reset indices
combinedPredictions.reset_index(drop=True,inplace=True)
combinedPredictionsTarget.reset_index(drop=True,inplace=True)

## Subset Q4 data and concat it with its level-0 predctions

In [210]:
#Get Q4 data
X_Test_Q4 = wd_h[(wd_h['dteday'] >= datetime.datetime(2012, 10, 1, 0, 0)) & (wd_h['dteday'] <= datetime.datetime(2012, 12, 31, 0, 0))].drop(['cnt','casual','registered','dteday'],axis=1)## NONE OF THEM IN DATA
Y_cnt_test_Q4 =wd_h['cnt'][(wd_h['dteday'] >= datetime.datetime(2012, 10, 1, 0, 0)) & (wd_h['dteday'] <= datetime.datetime(2012, 12, 31, 0, 0))]
oringal_cnt = Y_cnt_test_Q4

#get level-0 predictions for Q4 data
Q4_lm_predications = pd.DataFrame(lm.predict(X_Test_Q4), columns=["lm_predications"])
Q4_rf_predications = pd.DataFrame(rf.predict(X_Test_Q4), columns=["rf_predications"])

X_Test_Q4.reset_index(drop=True,inplace=True)
Q4_lm_predications.reset_index(drop=True,inplace=True)
Q4_rf_predications.reset_index(drop=True,inplace=True)
Y_cnt_test_Q4.reset_index(drop=True,inplace=True)

#concat Q4 with level-0 predictions 
X_Test_Q4_with_predictions = pd.concat([Q4_lm_predications, Q4_rf_predications, X_Test_Q4, ], axis=1)
Y_cnt_test_Q4 = pd.DataFrame({"target": Y_cnt_test_Q4})

## Q4 scores

In [196]:
lm.score(X_Test_Q4, Y_cnt_test_Q4)

0.840784756516971

In [197]:
rf.score(X_Test_Q4, Y_cnt_test_Q4)

0.8440622578283898

## Train and test Q4

In [198]:

RF_parameters = {'n_estimators': [10, 30 ,100],
                                             'bootstrap': [True],
                                             'max_depth': [80, 100 ],
                                             'max_features': ['sqrt',16],
                                             'min_samples_leaf': [2,  5 , 8],
                                             'min_samples_split': [ 10 , 8 , 15],
                                            'random_state':[random_seed],
                                            'criterion':['mse']}
rf_level_1 = GridSearchCV(RandomForestRegressor(),
                                 param_grid= RF_parameters,
                                 cv=tscv)

rf_level_1.fit(combinedPredictions, combinedPredictionsTarget)
rf_level_1.cv_results_
rf_level_1_predictions = lm.predict(X_Test_Q4)
rf_level_1.score(X_Test_Q4_with_predictions, Y_cnt_test_Q4)

0.8610394897985875

In [216]:
target = pd.DataFrame({"predicted_lvl_1_cnt": rf_level_1_predictions})

X_Test_Q4_with_lvl_0_pred_and_lvl_1_pred = pd.concat([X_Test_Q4_with_predictions, oringal_cnt, target], axis=1)
X_Test_Q4_with_lvl_0_pred_and_lvl_1_pred["diff"] = X_Test_Q4_with_lvl_0_pred_and_lvl_1_pred["lm_predications"] - X_Test_Q4_with_lvl_0_pred_and_lvl_1_pred["predicted_lvl_1_cnt"]
np.unique(X_Test_Q4_with_lvl_0_pred_and_lvl_1_pred["diff"] )


array([0.])

In [217]:
X_Test_Q4_with_lvl_0_pred_and_lvl_1_pred

,lm_predications,rf_predications,windspeed,yr,temp,RushHour-Low,hum,relative_windspeed,relative_hum,atemp,...,8,9,10,11,12,13,14,cnt,predicted_lvl_1_cnt,diff
0,79.762498,68.476071,0.1045,1,0.46,1,0.72,-0.784697,0.794349,0.4545,...,-1.0516,-1.7232,-1.7232,-2.6064,-1.7232,-1.7232,-1.783736,45,79.762498,0.0
1,55.535144,35.003026,0.0896,1,0.44,1,0.77,-0.925818,1.120130,0.4394,...,-0.9536,-1.5872,-1.5872,-2.4144,-1.5872,-1.5872,-1.619584,18,55.535144,0.0
2,60.004778,55.421885,0.0000,1,0.46,1,0.72,-1.808666,0.783031,0.4545,...,-1.0516,-1.7232,-1.7232,-2.6064,-1.7232,-1.7232,-1.783736,12,60.004778,0.0
3,46.681481,21.518511,0.0000,1,0.44,1,0.77,-1.784626,1.107291,0.4394,...,-0.9536,-1.5872,-1.5872,-2.4144,-1.5872,-1.5872,-1.619584,7,46.681481,0.0
4,38.163157,20.181529,0.1045,1,0.42,1,0.82,-0.742610,1.425880,0.4242,...,-0.8564,-1.4528,-1.4528,-2.2256,-1.4528,-1.4528,-1.459688,10,38.163157,0.0
5,66.936049,43.706550,0.0000,1,0.44,1,0.77,-1.758582,1.086114,0.4394,...,-0.9536,-1.5872,-1.5872,-2.4144,-1.5872,-1.5872,-1.619584,36,66.936049,0.0
6,124.164252,127.142689,0.1045,1,0.44,0,0.77,-0.742610,1.076992,0.4394,...,-1.9536,-2.5872,-2.5872,-3.4144,-2.5872,-2.5872,-2.619584,155,124.164252,0.0
7,386.404449,445.281732,0.1642,1,0.44,0,0.77,-0.169539,1.066383,0.4394,...,-6.8336,-7.9072,-7.9072,-11.6144,-7.9072,-7.9072,-12.526784,483,386.404449,0.0
8,599.208750,564.321479,0.1045,1,0.46,0,0.77,-0.756597,1.059487,0.4545,...,-8.4316,-9.5632,-9.5632,-13.3664,-9.5632,-9.5632,-15.018536,784,599.208750,0.0
9,308.827467,295.853456,0.0000,1,0.52,0,0.63,-1.772820,0.146093,0.5000,...,-2.8704,-4.3904,-4.3904,-6.9104,-4.3904,-4.3904,-6.390400,340,308.827467,0.0
